In [1]:
%load_ext autoreload
%autoreload 2

In [46]:
import os
import numpy as np
import pandas as pd

from scipy.stats import wilcoxon

In [3]:
os.chdir('..')

In [61]:
from evip2.utility.signiture import extract_sample_from_sigfile
from evip2.utility.kallisto import (
    concat_tpm_from_abundance,
    sum_tpm_by_gene_name,
    calculate_tpm_log2,
    calculate_tpm_zscore
)
from evip2.utility.evip import (
    calculate_corr_sample,
    # compare_functional_impact,
    calculate_null_distribution,
    calculate_wildtype_distribution
)
from gtfparse import read_gtf

In [6]:
sigfile = r'../tutorial_files/RNF43_sig.info'
gtffile = r'../tutorial_files/Homo_sapiens.GRCh38.104.gtf'

In [7]:
samples = extract_sample_from_sigfile(sigfile)

/home/byeungchun/workspace/eVIP2/evip2new/env/lib/python3.8/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [8]:
samples

['RNF43_WT_4',
 'RNF43_WT_3',
 'RNF43_WT_2',
 'RNF43_WT_1',
 'GFP_4',
 'GFP_3',
 'GFP_2',
 'GFP_1',
 'RNF43_R117fs_4',
 'RNF43_R117fs_3',
 'RNF43_R117fs_2',
 'RNF43_R117fs_1',
 'RNF43_G659fs_4',
 'RNF43_G659fs_3',
 'RNF43_G659fs_2',
 'RNF43_G659fs_1']

In [9]:
abundance_folder = r'../tutorial_files/RNF43_kallisto_outputs'
abundance_files = {x:f'{abundance_folder}/{x}/abundance.tsv' for x in samples}

In [10]:
df_tpm =concat_tpm_from_abundance(abundance_files)

In [11]:
df_gtf = read_gtf(gtffile)

/home/byeungchun/workspace/eVIP2/evip2new/env/lib/python3.8/site-packages/gtfparse/read_gtf.py:151: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  result = parse_gtf(
/home/byeungchun/workspace/eVIP2/evip2new/env/lib/python3.8/site-packages/gtfparse/read_gtf.py:151: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  result = parse_gtf(
INFO:root:Extracted GTF attributes: ['gene_id', 'gene_version', 'gene_name', 'gene_source', 'gene_biotype', 'transcript_id', 'transcript_version', 'transcript_name', 'transcript_source', 'transcript_biotype', 'tag', 'ccds_id', 'transcript_support_level', 'exon_number', 'exon_id', 'exon_version', 'protein_id', 'protein_version']


In [12]:
df_gtf.head()

,seqname,source,feature,start,end,score,strand,frame,gene_id,gene_version,...,transcript_source,transcript_biotype,tag,ccds_id,transcript_support_level,exon_number,exon_id,exon_version,protein_id,protein_version
0,1,ensembl_havana,gene,685679,686673,NaN,-,0,ENSG00000284662,1,...,,,,,,,,,,
1,1,ensembl_havana,transcript,685679,686673,NaN,-,0,ENSG00000284662,1,...,ensembl_havana,protein_coding,"CCDS,basic",CCDS41221,NA,,,,,
2,1,ensembl_havana,exon,685679,686673,NaN,-,0,ENSG00000284662,1,...,ensembl_havana,protein_coding,"CCDS,basic",CCDS41221,NA,1,ENSE00002324228,3,,
3,1,ensembl_havana,CDS,685719,686654,NaN,-,0,ENSG00000284662,1,...,ensembl_havana,protein_coding,"CCDS,basic",CCDS41221,NA,1,,,ENSP00000329982,2
4,1,ensembl_havana,start_codon,686652,686654,NaN,-,0,ENSG00000284662,1,...,ensembl_havana,protein_coding,"CCDS,basic",CCDS41221,NA,1,,,,


In [13]:
df_tpm_gene = sum_tpm_by_gene_name(df_tpm, df_gtf)

In [14]:
df_tpm_gene_log = calculate_tpm_log2(df_tpm_gene)

In [15]:
df_tpm_gene_zscore = calculate_tpm_zscore(df_tpm_gene_log)

In [16]:
df_sample_corr = calculate_corr_sample(df_tpm_gene_zscore)

In [17]:
df_sample_corr

,RNF43_WT_4,RNF43_WT_3,RNF43_WT_2,RNF43_WT_1,GFP_4,GFP_3,GFP_2,GFP_1,RNF43_R117fs_4,RNF43_R117fs_3,RNF43_R117fs_2,RNF43_R117fs_1,RNF43_G659fs_4,RNF43_G659fs_3,RNF43_G659fs_2,RNF43_G659fs_1
RNF43_WT_4,1.000000,0.342379,0.380222,0.278899,-0.290198,-0.227661,-0.264003,-0.226608,-0.180859,-0.109587,-0.115238,0.050217,0.008933,-0.090805,-0.091454,0.082259
RNF43_WT_3,0.342379,1.000000,0.213261,0.407289,-0.318379,-0.030391,-0.249859,-0.112808,-0.234877,-0.141598,-0.030476,0.156179,-0.110127,-0.178230,-0.213370,0.065399
RNF43_WT_2,0.380222,0.213261,1.000000,0.178751,-0.309090,-0.336316,-0.262289,-0.389742,-0.277084,0.019698,-0.072241,-0.135163,0.141349,0.105528,0.166766,0.165907
RNF43_WT_1,0.278899,0.407289,0.178751,1.000000,-0.273459,0.054098,-0.205463,0.003805,-0.287938,-0.172972,0.014445,0.320500,-0.226883,-0.292684,-0.306240,-0.012267
GFP_4,-0.290198,-0.318379,-0.309090,-0.273459,1.000000,0.340129,0.557652,0.252566,-0.149386,0.034766,-0.022762,-0.260479,-0.241272,-0.133771,-0.195955,-0.343583
GFP_3,-0.227661,-0.030391,-0.336316,0.054098,0.340129,1.000000,0.324981,0.372938,-0.307980,-0.121909,-0.026369,0.010313,-0.298959,-0.208771,-0.286390,-0.223951
GFP_2,-0.264003,-0.249859,-0.262289,-0.205463,0.557652,0.324981,1.000000,0.229603,-0.181720,0.009692,-0.023788,-0.219506,-0.283446,-0.164740,-0.205795,-0.282129
GFP_1,-0.226608,-0.112808,-0.389742,0.003805,0.252566,0.372938,0.229603,1.000000,0.099948,-0.275878,-0.156035,0.072390,-0.248908,-0.324490,-0.317049,-0.283222
RNF43_R117fs_4,-0.180859,-0.234877,-0.277084,-0.287938,-0.149386,-0.307980,-0.181720,0.099948,1.000000,-0.095993,-0.112158,0.041481,0.019937,-0.170010,-0.052439,-0.140470
RNF43_R117fs_3,-0.109587,-0.141598,0.019698,-0.172972,0.034766,-0.121909,0.009692,-0.275878,-0.095993,1.000000,0.443425,-0.153203,-0.181616,-0.089195,-0.115616,-0.213351


In [18]:
sig_id = {'GFP':{'type':'CT'}, 'RNF43_G659fs':{'type':'MT'}, 'RNF43_R117fs':{'type':'MT'}, 'RNF43_WT':{'type':'WT'}}

In [19]:
rep_null_dist, conn_null_dist = calculate_null_distribution(df_sample_corr, sig_id)

In [64]:
WT_dict, wt_rep_pvals, wt_ordered = calculate_wildtype_distribution(df_sample_corr, rep_null_dist)

In [26]:
gctx = r'../tutorial_files/eVIP2_out/eVIP_out/spearman_rank_matrix.gct'

In [30]:
cor_tbl = pd.read_csv(gctx, sep='\t', skiprows=2, index_col=0)

In [33]:
cor_tbl.sort_index()[cor_tbl.sort_index().index]

,GFP_1,GFP_2,GFP_3,GFP_4,RNF43_G659fs_1,RNF43_G659fs_2,RNF43_G659fs_3,RNF43_G659fs_4,RNF43_R117fs_1,RNF43_R117fs_2,RNF43_R117fs_3,RNF43_R117fs_4,RNF43_WT_1,RNF43_WT_2,RNF43_WT_3,RNF43_WT_4
id,,,,,,,,,,,,,,,,
GFP_1,1.000000,0.244672,0.364088,0.265590,-0.294024,-0.315762,-0.332628,-0.253749,0.067452,-0.150704,-0.258196,0.102746,0.009053,-0.392349,-0.111961,-0.226113
GFP_2,0.244672,1.000000,0.319373,0.558318,-0.286437,-0.214105,-0.179631,-0.286846,-0.219299,-0.024981,0.005284,-0.172081,-0.195334,-0.251432,-0.239337,-0.261213
GFP_3,0.364088,0.319373,1.000000,0.335566,-0.225191,-0.285372,-0.216213,-0.296453,0.011485,-0.037899,-0.121983,-0.291773,0.065091,-0.327385,-0.028304,-0.207214
GFP_4,0.265590,0.558318,0.335566,1.000000,-0.340412,-0.205012,-0.154968,-0.252790,-0.259765,-0.029560,0.034218,-0.138289,-0.256627,-0.309371,-0.306438,-0.287068
RNF43_G659fs_1,-0.294024,-0.286437,-0.225191,-0.340412,1.000000,0.331934,0.369220,0.250319,-0.037115,-0.201255,-0.208816,-0.150574,-0.006571,0.191899,0.069716,0.099409
RNF43_G659fs_2,-0.315762,-0.214105,-0.285372,-0.205012,0.331934,1.000000,0.577696,0.460132,-0.249194,-0.257324,-0.114184,-0.055516,-0.283405,0.180109,-0.187203,-0.080991
RNF43_G659fs_3,-0.332628,-0.179631,-0.216213,-0.154968,0.369220,0.577696,1.000000,0.414000,-0.255061,-0.230066,-0.083721,-0.167448,-0.271312,0.128907,-0.155031,-0.064897
RNF43_G659fs_4,-0.253749,-0.286846,-0.296453,-0.252790,0.250319,0.460132,0.414000,1.000000,-0.196760,-0.267330,-0.168833,0.009370,-0.205683,0.147225,-0.090144,0.011920
RNF43_R117fs_1,0.067452,-0.219299,0.011485,-0.259765,-0.037115,-0.249194,-0.255061,-0.196760,1.000000,0.065097,-0.141728,0.041963,0.310405,-0.116464,0.160559,0.052223
